In [129]:
from glob import glob
import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import cv2
import struct
import serial
import math

ser = None

In [122]:
def image_processing():#函数功能，将任意形状的png图片转化为20x20的0-1灰度图片并将其边缘扩充，最终生成28x28的tensor矩阵
    #  待处理图片路径
    img_path = Image.open('C:/temp/num.png')
    #  resize图片大小，入口参数为一个tuple，新的图片的大小
    img_size = img_path.resize((20, 20))
    #保存该图片
    img_size.save('C:/temp/num.png', 'PNG')
    #彩色形式打开
    png_img = cv2.imread(r'C:/temp/num.png')
    #  转化为灰度图像，此时仍未255表示形式
    gray_img=cv2.cvtColor(png_img,cv2.COLOR_RGB2GRAY)
    #  将255值转化为1值灰度图片
    gray_img_01=np.zeros_like(gray_img, dtype=float)
    for i in range(0,20):
        for j in range(0,20):
            gray_img_01[i][j]=(255.0-gray_img[i][j])/255.0
            
    #边缘黑色填充为28x28大小的图片
    img=np.zeros((28,28),dtype=float)
    for i in range(0,20):
        for j in range(0,20):
            img[i+4][j+4]=gray_img_01[i][j]
    img=torch.tensor(img,dtype=float)
    f=open("C:/temp/img.txt","w")
    for i in range(0,28):
        for j in range(0,28):
            f.write(str(float(img[i][j]))+'\n')
    f.close()
    return img

def trans_img_to_string(img):
    # print(img)
    msg = []
    for i in range(0,28):
        for j in range(0,28):
            tmp = struct.pack('!f',float(img[i][j]))
            # print(tmp)
            msg.append(tmp)
    return msg

def ser_init():
    global ser
    ser = serial.Serial(port="COM4",baudrate=115200,bytesize=serial.EIGHTBITS,
                        parity=serial.PARITY_NONE,stopbits=serial.STOPBITS_ONE,timeout=5,rtscts=False)
    if ser.isOpen():
        print("port COM4 open succeeded")
    else:
        print("port COM4 open failed")

def ser_final():
    global ser
    print("port COM4 closed")
    ser.close()

def print_hex(num):
    print('32\'h',end='')
    for i in range(4):
        print('%02x'%((struct.pack('!f',float(num)))[i]),end='')
    print(';')

In [136]:
display_list = [0,100,116,135,200, 208,300,350,400,450, 500,550,600,700,750]
def ser_send():
    global ser
    send_len = 0
    img = image_processing()
    msg = trans_img_to_string(img)
    for i in range(784):
        send_len += ser.write(msg[i])
        
    read_msg = ser.read(40)

    f=open("C:/temp/data.txt","w")
    res = []
    for i in range(10):
        res.append((struct.unpack('!f',read_msg[i*4:(i+1)*4]))[0])

    prob = []
    sum=0.0
    for i in range(10):
        sum+=math.exp(res[i])
    for i in range(10):
        prob.append(math.exp(res[i])/sum)

    for num in prob:
        f.write(str(num)+'\n')
    f.close()

In [96]:
for i,num in enumerate(display_list):
    print('\t\t\tdisplaySel[',i,']',' ? 32\'d',(num+30)*4,' :',sep='')

			displaySel[0] ? 32'd120 :
			displaySel[1] ? 32'd520 :
			displaySel[2] ? 32'd584 :
			displaySel[3] ? 32'd660 :
			displaySel[4] ? 32'd920 :
			displaySel[5] ? 32'd952 :
			displaySel[6] ? 32'd1320 :
			displaySel[7] ? 32'd1520 :
			displaySel[8] ? 32'd1720 :
			displaySel[9] ? 32'd1920 :
			displaySel[10] ? 32'd2120 :
			displaySel[11] ? 32'd2320 :
			displaySel[12] ? 32'd2520 :
			displaySel[13] ? 32'd2920 :
			displaySel[14] ? 32'd3120 :


In [60]:
b1 = b'\x68\x4b\x4b\xef'
b2 = b'\xbe\x33\xa8\x17'
n1 = struct.unpack('!f',b1)[0]
n2 = struct.unpack('!f',b2)[0]
print(n1)
print(n2)
print(n1+n2)
print_hex(n1+n2)

3.840164494372165e+24
-0.17544589936733246
3.840164494372165e+24
32'h684b4bef;


In [73]:
n1 = [0.0052766679,-0.0170026999,-0.0158943646,0.0087642977,1.9423172474]
n2 = [-1.9890792370,-0.0010372525,0.0077704308,-0.6294782162,-0.3872341514]
result = [0,0,0,0,0]
for i in range(len(n1)):
    print('#5')
    print('n1=',end='')
    print_hex(n1[i])
    print('n2=',end='')
    print_hex(n2[i])
for i in range(len(n1)):
    result[i]=n1[i]*n2[i]
    print_hex(result[i])


#5
n1=32'h3bace7e6;
n2=32'hbffe9a26;
#5
n1=32'hbc8b493f;
n2=32'hba87f46b;
#5
n1=32'hbc8234e6;
n2=32'h3bfe9f19;
#5
n1=32'h3c0f9821;
n2=32'hbf21257c;
#5
n1=32'h3ff89dda;
n2=32'hbec6438e;
32'hbc2bf633;
32'h3793f139;
32'hb9018168;
32'hbbb4c767;
32'hbf408bb2;


In [132]:
ser_init()

port COM4 open succeeded


In [137]:
ser_send()

[-1.8425252437591553, 1.9275861978530884, -0.8280611038208008, 3.8190860748291016, -1.4746131896972656, 1.661027431488037, 0.5360004901885986, -0.8458044528961182, -0.17075121402740479, -2.9481570720672607]
0.002573449581152067
0.11164885131689697
0.00709727841201735
0.740154217830689
0.0037179011513452185
0.0855242467636548
0.027764909697962387
0.006972459550654973
0.013694869561057012
0.0008518161345702092


In [120]:
ser_final()

port COM4 closed
